In [1]:
# !pip install -q torch transformers accelerate transformers sentence-transformers faiss-cpu

In [2]:
# !pip install -q langchain langchain-community

In [1]:
from langchain_community.document_loaders import PyPDFLoader
import os


# loop through PDFs and load them with PyPDFLoader
file_path = "../pdfs"
docs = []
for file in os.listdir(file_path):
    if file.endswith('.pdf'):
        pdf_path = os.path.join(file_path, file)
        loader = PyPDFLoader(pdf_path)
        docs.extend(loader.load())

docs

/Users/rickchakra/Projects/aiug_workshop/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong pointing object 29 0 (offset 0)
Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 11 0 (offset 0)
Ignoring wrong

[Document(metadata={'source': '../pdfs/paleo_internal_plan.pdf', 'page': 0}, page_content='Paleo Diet Nutrition Plan The Paleo diet is beneficial for: • Weight loss: Promotes lean body mass and reduces body fat • Improved insulin sensitivity: May help manage or prevent type 2 diabetes • Reduced inflammation: Can alleviate symptoms of autoimmune diseases • Better digestion: Eliminates common gut irritants • Increased nutrient density: Focuses on whole, nutrient-rich foods • Improved energy levels: Stable blood sugar leads to consistent energy • Better sleep: May improve sleep quality due to reduced inflammation This diet is ideal for: • Individuals looking to lose weight or maintain a healthy weight • People with autoimmune conditions or inflammatory disorders • Those seeking to improve overall health and energy levels • Individuals with digestive issues or food sensitivities • People interested in a whole-food based approach to eating • Athletes and fitness enthusiasts looking for perf

In [2]:
# inspect the content of a document
docs[2].page_content

'Paleo Lunch Options:  a) Grilled chicken salad: • 4 oz grilled chicken breast • 2 cups mixed greens • 1/4 cup sliced cucumbers • 1/4 cup cherry tomatoes • 1/4 avocado, diced • 2 tbsp pumpkin seeds • Dressing: 1 tbsp olive oil, 1 tsp lemon juice, herbs b) Tuna lettuce wraps: • 4 oz canned tuna (in water) • 1 tbsp paleo-friendly mayonnaise • 1 tbsp diced celery • 1 tbsp diced red onion • 1 tsp dijon mustard • Serve in 3-4 large lettuce leaves c) Beef and vegetable stir-fry: • 4 oz grass-fed beef strips • 1 cup mixed vegetables (broccoli, carrots, snap peas) • 1 tbsp coconut aminos • 1 tbsp coconut oil for cooking • 1 clove garlic, minced • Serve over cauliflower rice         '

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# split documents into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=800, chunk_overlap=0)

chunked_docs = splitter.split_documents(docs)

In [3]:
# inspect a chunk
chunked_docs[2]

Document(metadata={'source': '../pdfs/paleo_internal_plan.pdf', 'page': 1}, page_content='Paleo Breakfast Options:  a) Paleo breakfast bowl: • 2 eggs, fried or poached • 1/4 avocado, sliced • 1/2 cup sautéed mixed vegetables (spinach, mushrooms, bell peppers) • 2 slices of bacon (uncured, no added nitrates) • 1/4 cup berries on the side b) Sweet potato hash: • 1 small sweet potato, diced and roasted • 3 oz ground turkey or chicken, seasoned and cooked • 1/4 cup diced onions • 1/4 cup diced bell peppers • 1 tbsp coconut oil for cooking • Herbs: rosemary, thyme c) Paleo smoothie: • 1 cup unsweetened almond milk • 1/2 banana • 1 tbsp almond butter • 1 scoop paleo-friendly protein powder • 1 cup spinach • 1/2 cup mixed berries • Ice cubes')

In [4]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# initialize vectorstore with FAISS and HuggingFace embeddings
db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))

/Users/rickchakra/Projects/aiug_workshop/venv/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/rickchakra/Projects/aiug_workshop/venv/lib/python3.9/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [15]:
# create a retriever using the vectorstore
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [17]:
# test the retriever
test = retriever.invoke('Breakfast options')
print(len(test))

print(test)

3
[Document(metadata={'source': '../pdfs/vegan_internal_plan.pdf', 'page': 1}, page_content='Vegan Breakfast Options:  a) Tofu scramble: • 1/2 block firm tofu, crumbled • 1/4 cup mixed vegetables (bell peppers, onions, spinach) • 1 tbsp nutritional yeast • 1/4 tsp turmeric • Salt and pepper to taste • Serve with 1 slice whole grain toast b) Overnight oats: • 1/2 cup rolled oats • 3/4 cup plant-based milk (almond, soy, oat) • 1 tbsp chia seeds • 1 tbsp maple syrup • 1/4 cup mixed berries • 1 tbsp chopped nuts c) Vegan breakfast smoothie: • 1 banana • 1 cup spinach • 1 tbsp almond butter • 1 scoop vegan protein powder • 1 cup plant-based milk • 1/2 tsp cinnamon'), Document(metadata={'source': '../pdfs/ketogenic_internal_plan.pdf', 'page': 1}, page_content='Detailed Meal Plan Ketogenic Breakfast Options:  a) Keto breakfast bowl: • 2 scrambled eggs • 1/4 avocado, sliced • 2 strips of bacon • 1/4 cup sautéed spinach • 1 tbsp butter for cooking b) Cream cheese pancakes: • 2 oz cream cheese •

In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name ="TinyLlama/TinyLlama-1.1B-Chat-v1.0"

save_directory = "model_directory"

# directly load model since we saved it in the previous notebook
model = AutoModelForCausalLM.from_pretrained(save_directory)
tokenizer = AutoTokenizer.from_pretrained(save_directory)

In [9]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

# create a text generation pipeline
text_generation_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=400,
)

llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | llm | StrOutputParser()

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [10]:
from langchain_core.runnables import RunnablePassthrough

# extend llm chain with RAG
rag_chain = {"context": retriever, "question": RunnablePassthrough()} | llm_chain

In [11]:
question = "What are some ketogenic breakfast options?"

In [12]:
# invoke LLM chain for baseline response
result = llm_chain.invoke({"context": "", "question": question})

result

'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nWhat are some ketogenic breakfast options?\n</s>\n<|assistant|>\n\n 1. Greek yogurt with berries and granola: This is a great option for a protein-packed breakfast that also provides healthy fats from the Greek yogurt and fiber from the berries. You can add a tablespoon of honey or maple syrup for sweetness.\n\n2. Smoothie: Blend together frozen banana, spinach, almond milk, and a scoop of protein powder for a nutrient-dense breakfast smoothie. You can also add in some chia seeds for extra fiber and omega-3s.\n\n3. Overnight oats: Mix rolled oats, almond milk, chia seeds, and a sprinkle of cinnamon in a jar or container overnight in the fridge. In the morning, top with fresh fruit and nuts.\n\n4. Avocado toast: Toast a slice of whole grain bread and spread mashed avocado on top. Add sliced tomatoes, red onions, and a sprinkle of salt and pepper.\n\n5. Protein pancakes: 

In [13]:
# invoke RAG chain for response based on knowledge base
result = rag_chain.invoke(question)

result

"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(metadata={'source': '../pdfs/ketogenic_internal_plan.pdf', 'page': 1}, page_content='Detailed Meal Plan Ketogenic Breakfast Options:  a) Keto breakfast bowl: • 2 scrambled eggs • 1/4 avocado, sliced • 2 strips of bacon • 1/4 cup sautéed spinach • 1 tbsp butter for cooking b) Cream cheese pancakes: • 2 oz cream cheese • 2 eggs • 1/4 tsp cinnamon • 1 tbsp butter for cooking • Top with 2 tbsp sugar-free syrup c) Keto smoothie: • 1 cup unsweetened almond milk • 1/2 avocado • 1 tbsp almond butter • 1 scoop low-carb protein powder • 1 cup spinach • Ice cubes')]\n\n</s>\n<|user|>\nWhat are some ketogenic breakfast options?\n</s>\n<|assistant|>\n\n 1. Keto breakfast bowl:\n    - 2 scrambled eggs\n    - 1/4 avocado, sliced\n    - 2 strips of bacon\n    - 1/4 cup sautéed spinach\n    - 1 tablespoon butter for cooking\n\n2. Cream cheese pancakes:\n    - 2 oz cream cheese\n    - 2 eggs\n    

In [14]:
from pprint import pprint

pprint(result)

('\n'
 '<|system|>\n'
 'Answer the question based on your knowledge. Use the following context to '
 'help:\n'
 '\n'
 "[Document(metadata={'source': '../pdfs/ketogenic_internal_plan.pdf', 'page': "
 "1}, page_content='Detailed Meal Plan Ketogenic Breakfast Options:  a) Keto "
 'breakfast bowl: • 2 scrambled eggs • 1/4 avocado, sliced • 2 strips of bacon '
 '• 1/4 cup sautéed spinach • 1 tbsp butter for cooking b) Cream cheese '
 'pancakes: • 2 oz cream cheese • 2 eggs • 1/4 tsp cinnamon • 1 tbsp butter '
 'for cooking • Top with 2 tbsp sugar-free syrup c) Keto smoothie: • 1 cup '
 'unsweetened almond milk • 1/2 avocado • 1 tbsp almond butter • 1 scoop '
 "low-carb protein powder • 1 cup spinach • Ice cubes')]\n"
 '\n'
 '</s>\n'
 '<|user|>\n'
 'What are some ketogenic breakfast options?\n'
 '</s>\n'
 '<|assistant|>\n'
 '\n'
 ' 1. Keto breakfast bowl:\n'
 '    - 2 scrambled eggs\n'
 '    - 1/4 avocado, sliced\n'
 '    - 2 strips of bacon\n'
 '    - 1/4 cup sautéed spinach\n'
 '    - 1 ta